## Regressão Linear Múltipla

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import scipy.stats as stats
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.anova import anova_lm

import matplotlib.pyplot as plt
import seaborn as sns

import graphs

In [ ]:
plt.rcParams['figure.figsize'] = [12, 8]

### Expectativa de Vida

Temos um dataset com os seguintes atributos:

- Population: population estimate as of July 1, 1975
- Income: per capita income (1974)
- Illiteracy: illiteracy (1970, percent of population)
- Life Exp: life expectancy in years (1969–71)
- Murder: murder and non-negligent manslaughter rate per 100,000 population (1976)
- HS Grad: percent high-school graduates (1970)
- Frost: mean number of days with minimum temperature below freezing (1931–1960) in capital or large city
- Area: land area in square miles

In [ ]:
states = pd.read_csv('states.csv', index_col=0)

In [ ]:
states.head()

In [ ]:
states.rename(columns={'Life Exp': 'LifeExp', 'HS Grad': 'HSGrad'}, inplace=True)

In [ ]:
states['Density'] = states['Population'] * 1000 / states['Area']

In [ ]:
states.sample(10)

In [ ]:
states.describe()

In [ ]:
sns.heatmap(states.corr(), annot=True, cmap="viridis", fmt="0.2f");

In [ ]:
pd.plotting.scatter_matrix(states);

Podemos estimar a expectativa de vida baseado no estado em que ele reside?

Criando um modelo com todas as variáveis

In [ ]:
X = sm.add_constant(states.drop('LifeExp', axis=1))
y = states['LifeExp']
X.head()

In [ ]:
model = sm.OLS(y, X).fit()

In [ ]:
model.summary()

Muitas variáveis preditoras não são significativas, porém a regressão no geral é significativa

In [ ]:
graph_plotter = graphs.AssumptionGraphs(model)

In [ ]:
graph_plotter.plot_residual_fitted_values(y)

In [ ]:
graph_plotter.plot_qq()

In [ ]:
graph_plotter.plot_scale_location()

In [ ]:
graph_plotter.plot_influence()

In [ ]:
model.get_influence().plot_influence();

#### Fator de inflação da variância

In [ ]:
pd.Series([variance_inflation_factor(X.values, i) 
           for i in range(X.shape[1])], 
           index=X.columns)

#### Gráfico de regressão parcial ou de variável agregada

In [ ]:
fig = plt.figure(figsize=(12, 16))
sm.graphics.plot_partregress_grid(model, fig=fig);

Padrões distintos são indicadores de boas contribuições para o modelo. Enquanto que a falta de padrões geralmente apontam para variaveis que podem ser removidas do modelo.

Como a variável `Illiteracy` tem um alto fator de inflação de variância, um p-valor insignificante no modelo como um todo e não tem um padrão distinto nos gráficos de regressão parcial. O que vai acontecer se removermos essa variável? 

In [ ]:
X = sm.add_constant(states.drop(['LifeExp', 'Illiteracy'], axis=1))
y = states['LifeExp']
X.head()

In [ ]:
model2 = sm.OLS(y, X).fit()

In [ ]:
model2.summary()

O $R^2$ ajustado aumentou e o modelo continua significativo

In [ ]:
graph_plotter = graphs.AssumptionGraphs(model2)

In [ ]:
graph_plotter.plot_residual_fitted_values(y)

In [ ]:
graph_plotter.plot_qq()

In [ ]:
graph_plotter.plot_scale_location()

In [ ]:
graph_plotter.plot_influence()

In [ ]:
model2.get_influence().plot_influence();

Podemos perceber que a influência do `Hawaii` diminuiu

#### Fator de influência da variância

In [ ]:
pd.Series([variance_inflation_factor(X.values, i) 
           for i in range(X.shape[1])], 
           index=X.columns)

Todos os fatores diminuíram

Podemos comparar os dois modelos usando o F-test parcial usando a função anova

In [ ]:
anova_results = anova_lm(model2, model)
anova_results

O p-valor é bem alto, indicando que devemos ficar com a hipótese nula. Relembrando que a hipótese nula é que os coeficientes das variáveis no subconjunto de interesse são todas zero. Nós retemos a hipótese nula e concluímos que a variável `Illiteracy` não é informativa para o modelo.

Vamos usar o teste F parcial para testar múltiplas variáveis preditoras de uma vez. Comparando com o modelo saturado, o subconjunto `Illiteracy`, `Area` e `Income` tem algum efeito na previsão da expectativa de vida? 

In [ ]:
X = sm.add_constant(states.drop(['LifeExp'], axis=1))
y = states['LifeExp']
model_full = sm.OLS(y, X).fit()

In [ ]:
X = sm.add_constant(states.drop(['LifeExp', 'Illiteracy', 'Area', 'Income'], axis=1))
y = states['LifeExp']
model_reduced = sm.OLS(y, X).fit()

In [ ]:
anova_results = anova_lm(model_reduced, model_full)
anova_results

O p-valor é grande, então o modelo reduzido é suficiente

In [ ]:
model_reduced.summary()

In [ ]:
graph_plotter = graphs.AssumptionGraphs(model_reduced)

In [ ]:
graph_plotter.plot_residual_fitted_values(y)

In [ ]:
graph_plotter.plot_qq()

In [ ]:
graph_plotter.plot_scale_location()

In [ ]:
graph_plotter.plot_influence()

In [ ]:
model_reduced.get_influence().plot_influence();

In [ ]:
pd.Series([variance_inflation_factor(X.values, i) 
           for i in range(X.shape[1])], 
           index=X.columns)

In [ ]:
fig = plt.figure(figsize=(12, 16))
sm.graphics.plot_partregress_grid(model_reduced, fig=fig);

#### Métricas AIC e BIC

In [ ]:
pd.DataFrame({'Model': ['Full', 'Without Illiteracy', 'Reduced'], 'AIC': [model_full.aic, model2.aic, model_reduced.aic],
              'BIC': [model_full.bic, model2.bic, model_reduced.bic]})

In [ ]:
from model_selection import forward_selected, backward_selected

In [ ]:
states.columns

In [ ]:
best_model_forward = forward_selected(states, 'LifeExp')

In [ ]:
best_model_backward = backward_selected(states, 'LifeExp')

In [ ]:
best_model_forward.summary()

In [ ]:
graph_plotter = graphs.AssumptionGraphs(best_model_forward)

In [ ]:
graph_plotter.plot_residual_fitted_values(y)

In [ ]:
graph_plotter.plot_qq()

In [ ]:
graph_plotter.plot_scale_location()

In [ ]:
graph_plotter.plot_influence()

In [ ]:
X = sm.add_constant(states[['Murder', 'HSGrad', 'Frost', 'Population', 'Density']])
pd.Series([variance_inflation_factor(X.values, i) 
           for i in range(X.shape[1])], 
           index=X.columns)

In [ ]:
fig = plt.figure(figsize=(12, 16))
sm.graphics.plot_partregress_grid(best_model_forward, fig=fig);

### Nota na prova

In [ ]:
tests = pd.read_csv('test_scores.csv', sep=' ')
tests.head()

In [ ]:
tests.describe().T

In [ ]:
tests.corr()

In [ ]:
plt.scatter(tests['Hours.Studied'], tests['Test.Score'])
plt.title('Gráfico de Dispersão')
plt.xlabel('Horas Estudadas')
plt.ylabel('Nota na prova');

Ajustar um modelo de regressão linear simples

In [ ]:
X = sm.add_constant(tests['Hours.Studied'])
y = tests['Test.Score']
model = sm.OLS(y, X).fit()

In [ ]:
model.summary()

In [ ]:
graph_plotter = graphs.AssumptionGraphs(model)

In [ ]:
graph_plotter.plot_residual_fitted_values(y)

In [ ]:
graph_plotter.plot_qq()

In [ ]:
graph_plotter.plot_scale_location()

In [ ]:
graph_plotter.plot_influence()

In [ ]:
model.get_influence().plot_influence();

Vamos incluir um termo quadrático

In [ ]:
tests['Hours.Studied.Quadratic'] = tests['Hours.Studied'] ** 2

In [ ]:
X = sm.add_constant(tests[['Hours.Studied', 'Hours.Studied.Quadratic']])
y = tests['Test.Score']
model = sm.OLS(y, X).fit()

In [ ]:
model.summary()

In [ ]:
graph_plotter = graphs.AssumptionGraphs(model)

In [ ]:
graph_plotter.plot_residual_fitted_values(y)

In [ ]:
graph_plotter.plot_qq()

In [ ]:
graph_plotter.plot_scale_location()

In [ ]:
graph_plotter.plot_influence()

In [ ]:
model.get_influence().plot_influence();

Adicionando um fator

In [ ]:
dummy = sm.categorical(tests['Gender'], drop=True)
dummy.sample(10)

In [ ]:
X = sm.add_constant(np.column_stack((tests['Hours.Studied'].values, dummy.iloc[:, 1:])))
y = tests['Test.Score']
model = sm.OLS(y, X).fit()

In [ ]:
model.summary()

In [ ]:
graph_plotter = graphs.AssumptionGraphs(model)

In [ ]:
graph_plotter.plot_residual_fitted_values(y)

In [ ]:
graph_plotter.plot_qq()

In [ ]:
graph_plotter.plot_scale_location()

In [ ]:
graph_plotter.plot_influence()

In [ ]:
model.get_influence().plot_influence();

In [ ]:
categories = np.unique(tests['Gender'])
colors = {'Female': 'red', 'Male': 'black'}

plt.scatter(tests['Hours.Studied'], tests['Test.Score'], c=tests['Gender'].apply(lambda x: colors[x]))
plt.plot(tests['Hours.Studied'], tests['Hours.Studied'] * model.params['x1'] + model.params['const'], 
         color='red', label='Regressão Feminina')
plt.plot(tests['Hours.Studied'], tests['Hours.Studied'] * model.params['x1'] + model.params['const'] + model.params['x2'], 
         color='black', label='Regressão Masculina')
plt.title('Gráfico de Dispersão')
plt.xlabel('Horas Estudadas')
plt.ylabel('Nota na prova')
plt.legend();